In [1]:
!git clone https://github.com/Jackmin801/StackOverflowMine.git

Cloning into 'StackOverflowMine'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 14 (delta 1), reused 10 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), done.


In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('StackOverflowMine/JavaTop50000.csv')

In [4]:
data.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,LastEditorUserId,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,ContentLicense,PostId,TagId,Id.1,TagName,Count,ExcerptPostId,WikiPostId
0,11227809,1,11227902.0,NaN,2012-06-27 13:51:36,NaN,24856,1528592,<p>Here is a piece of C++ code that shows some...,87234.0,NaN,224132.0,NaN,2020-07-16 03:35:56,2020-09-15 02:13:25,Why is processing a sorted array faster than p...,<java><c++><performance><optimization><branch-...,26,2,11113.0,NaN,NaN,CC BY-SA 4.0,11227809,17,17,java,1715860,3624966,3607018
1,6841333,1,6841479.0,NaN,2011-07-27 08:15:58,NaN,6935,667557,"<p>If I run the following program, which parse...",342235.0,NaN,13602076.0,NaN,2020-07-14 19:07:57,2020-07-14 19:07:57,Why is subtracting these two times (in 1927) g...,<java><date><timezone>,10,5,1626.0,NaN,NaN,CC BY-SA 4.0,6841333,17,17,java,1715860,3624966,3607018
2,40480,1,NaN,NaN,2008-09-02 20:14:29,NaN,6724,2024594,<p>I always thought Java uses <strong>pass-by-...,4315.0,NaN,9622275.0,user4642421,2020-08-06 07:24:06,2020-09-16 15:00:21,"Is Java ""pass-by-reference"" or ""pass-by-value""?",<java><methods><parameter-passing><pass-by-ref...,81,1,3053.0,NaN,2012-09-14 18:22:55,CC BY-SA 4.0,40480,17,17,java,1715860,3624966,3607018
3,309424,1,309448.0,NaN,2008-11-21 16:47:40,NaN,4156,2158738,<p>If you have a <code>java.io.InputStream</co...,16616.0,Johnny Maelstrom,63550.0,NaN,2019-01-05 10:28:26,2020-04-05 00:03:47,How do I read / convert an InputStream into a ...,<java><string><io><stream><inputstream>,59,0,1193.0,NaN,NaN,CC BY-SA 4.0,309424,17,17,java,1715860,3624966,3607018
4,271526,1,271874.0,NaN,2008-11-07 08:31:40,NaN,4081,1267072,<p>I use <code>object != null</code> a lot to ...,34856.0,Goran Martinic,11880324.0,cletus,2020-08-17 13:56:41,2020-08-17 13:56:41,How to avoid null checking in Java?,<java><object><nullpointerexception><null>,62,30,1492.0,NaN,2011-10-21 09:02:37,CC BY-SA 4.0,271526,17,17,java,1715860,3624966,3607018


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Id                     50000 non-null  int64  
 1   PostTypeId             50000 non-null  int64  
 2   AcceptedAnswerId       40342 non-null  float64
 3   ParentId               0 non-null      float64
 4   CreationDate           50000 non-null  object 
 5   DeletionDate           0 non-null      float64
 6   Score                  50000 non-null  int64  
 7   ViewCount              50000 non-null  int64  
 8   Body                   50000 non-null  object 
 9   OwnerUserId            48894 non-null  float64
 10  OwnerDisplayName       3204 non-null   object 
 11  LastEditorUserId       37621 non-null  float64
 12  LastEditorDisplayName  1895 non-null   object 
 13  LastEditDate           38097 non-null  object 
 14  LastActivityDate       50000 non-null  object 
 15  Ti

In [14]:
clusters = pd.read_excel('Cluster100B.xlsx', sheet_name=None, index_col=0)
clusters = pd.concat(clusters, ignore_index=True)

In [19]:
del clusters['Title']
del clusters['ViewCount']

In [24]:
data = pd.merge(data, clusters, how='inner', on='Id')

In [26]:
data[['Title','Body','labels']].head()

,Title,Body,labels
0,Why is processing a sorted array faster than p...,<p>Here is a piece of C++ code that shows some...,67
1,Why is subtracting these two times (in 1927) g...,"<p>If I run the following program, which parse...",66
2,"Is Java ""pass-by-reference"" or ""pass-by-value""?",<p>I always thought Java uses <strong>pass-by-...,19
3,How do I read / convert an InputStream into a ...,<p>If you have a <code>java.io.InputStream</co...,87
4,How to avoid null checking in Java?,<p>I use <code>object != null</code> a lot to ...,3


In [28]:
data[['Title','Body','labels']][data['labels']==1]

,Title,Body,labels
166,Can I add jars to maven 2 build classpath with...,<p>Maven2 is driving me crazy during the exper...,1
678,How do I set the proxy to be used by the JVM,"<p>Many times, a Java app needs to connect to ...",1
787,Maven parent pom vs modules pom,<p>There seem to be several ways to structure ...,1
885,What's the best way to validate an XML file ag...,<p>I'm generating some xml files that needs to...,1
967,"In Java, how do I parse XML as a String instea...",<p>I have the following code: </p>\n\n<pre><co...,1
...,...,...,...
49341,What is the correct META-INF directory locatio...,"<p>Should a ""context.xml"" file for deploying a...",1
49413,Convert ivy.xml to pom.xml,"<p>I have a ivy.xml - <a href=""https://gist.gi...",1
49698,What is the difference between Castor XML bind...,<p>What is the difference between Castor XML a...,1
49913,Are there faster XML parsers in Java than Xala...,<p>I haven't found many ways to increase the p...,1


In [6]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [29]:
titles = [word_tokenize(str.lower(title)) for title in data.Title.values]

In [30]:
from collections import Counter


# Create a Counter with the lowercase tokens: bow_simple
bow_simple = Counter(titles[0])

# Print the 10 most common tokens
print(bow_simple.most_common(10))

[('processing', 2), ('array', 2), ('why', 1), ('is', 1), ('a', 1), ('sorted', 1), ('faster', 1), ('than', 1), ('an', 1), ('unsorted', 1)]


In [31]:
# Import Dictionary
from gensim.corpora.dictionary import Dictionary

# Create a Dictionary from the articles: dictionary
dictionary = Dictionary(titles)

# Use computer_id with the dictionary to print the word
print(dictionary.token2id)



{'?': 0, 'a': 1, 'an': 2, 'array': 3, 'faster': 4, 'is': 5, 'processing': 6, 'sorted': 7, 'than': 8, 'unsorted': 9, 'why': 10, '(': 11, ')': 12, '1927': 13, 'giving': 14, 'in': 15, 'result': 16, 'strange': 17, 'subtracting': 18, 'these': 19, 'times': 20, 'two': 21, "''": 22, '``': 23, 'java': 24, 'or': 25, 'pass-by-reference': 26, 'pass-by-value': 27, '/': 28, 'convert': 29, 'do': 30, 'how': 31, 'i': 32, 'inputstream': 33, 'into': 34, 'read': 35, 'string': 36, 'avoid': 37, 'checking': 38, 'null': 39, 'to': 40, 'and': 41, 'are': 42, 'between': 43, 'differences': 44, 'hashmap': 45, 'hashtable': 46, 'the': 47, 'what': 48, "'s": 49, '*=': 50, '+=': 51, ',': 52, '-=': 53, '/=': 54, 'assignment': 55, 'casting': 56, 'compound': 57, "n't": 58, 'operators': 59, 'require': 60, 'android': 61, 'cases': 62, 'for': 63, 'proper': 64, 'use': 65, 'usermanager.isuseragoat': 66, 'arraylist': 67, 'create': 68, 'from': 69, 'generate': 70, 'integers': 71, 'random': 72, 'range': 73, 'specific': 74, 'within':

In [32]:
corpus = [dictionary.doc2bow(title) for title in titles]

# Print the first 10 word ids with their frequency counts from the fifth document
print(corpus[4][:10])

[(0, 1), (15, 1), (24, 1), (31, 1), (37, 1), (38, 1), (39, 1), (40, 1)]


In [33]:
from gensim.models.tfidfmodel import TfidfModel
# Create a new TfidfModel using the corpus: tfidf
tfidf = TfidfModel(corpus)

In [34]:
tfidf[[(0,1),(15,1)]]

[(0, 0.6130057486250556), (15, 0.7900784468346388)]

In [ ]:
# Calculate the tfidf weights of doc: tfidf_weights
tfidf_weights = tfidf[corpus[140]]

# Print the first five weights
print(tfidf_weights[:1])

# Sort the weights from highest to lowest: sorted_tfidf_weights
sorted_tfidf_weights = sorted(tfidf_weights, key=lambda w: w[1], reverse=True)

# Print the top 5 weighted words
for term_id, weight in sorted_tfidf_weights[:10]:
    print(dictionary.get(term_id), weight)

[(0, 0.07527069981792461)]
dog 0.3975690069608745
animal 0.38165054812350707
< 0.33838135249497736
implicitly 0.3343828766427354
> 0.3290292942505539
list 0.3022991036238914
polymorphic 0.30168307258173266
subclass 0.2508290316176677
generics 0.20542780575024586
are 0.1522344543420726


In [43]:
group_1 = data[data['labels']==1].index.values

In [49]:
from collections import defaultdict
meow = defaultdict(lambda: 0)
for i in group_1:
  tfidf_weights = tfidf[corpus[i]]
  sorted_tfidf_weights = sorted(tfidf_weights, key=lambda w: w[1], reverse=True)
  for term_id, weight in sorted_tfidf_weights[:5]:
    meow[term_id] += 1


In [59]:
top_10 = sorted(meow.items(), key=lambda item: item[1], reverse=True)[:10]
for i in top_10:
  print(dictionary.get(i[0]),i[1])

xml 78
jaxb 28
file 19
schema 15
string 14
in 14
object 12
validate 11
to 11
xsd 10


In [96]:
from collections import defaultdict
def get_top_10_tfidf(n):
  group = data[data['labels']==n].index.values
  meow = defaultdict(lambda: 0)
  for i in group:
    tfidf_weights = tfidf[corpus[i]]
    sorted_tfidf_weights = sorted(tfidf_weights, key=lambda w: w[1], reverse=True)
    for term_id, weight in sorted_tfidf_weights[:5]:
      meow[term_id] += 1
  top_10 = sorted(meow.items(), key=lambda item: item[1], reverse=True)[:10]
  ret=[]
  for i in top_10:
    #print(dictionary.get(i[0]),i[1])
    ret.append((dictionary.get(i[0]),i[1]/len(group), len(group)))
  return ret

In [97]:
highest = [ranks[0] for ranks in [get_top_10_tfidf(i) for i in range(100)]]

In [98]:
for i,h in enumerate(highest):
  print(i,h)


0 ('static', 0.2183406113537118, 458)
1 ('xml', 0.25, 312)
2 ('android', 0.25041736227045075, 599)
3 (')', 0.051601423487544484, 562)
4 ('bigdecimal', 0.717948717948718, 117)
5 ('spring', 0.2961082910321489, 591)
6 (':', 0.06736526946107785, 668)
7 ('idea', 0.14634146341463414, 82)
8 ('int', 0.09984639016897082, 651)
9 ('enum', 0.6907216494845361, 291)
10 ('jpa', 0.141747572815534, 515)
11 ('java', 0.06787878787878789, 825)
12 ('maven', 0.42524916943521596, 301)
13 ('junit', 0.3145631067961165, 515)
14 ('stream', 0.4093567251461988, 342)
15 ('server', 0.17402597402597403, 385)
16 ('date', 0.35507246376811596, 276)
17 ('method', 0.059322033898305086, 472)
18 ('file', 0.34221598877980364, 713)
19 ('in', 0.05851755526657997, 769)
20 ('exception', 0.2702702702702703, 407)
21 ('intellij', 0.44680851063829785, 235)
22 ('thread', 0.3161904761904762, 525)
23 ('node', 0.1925925925925926, 135)
24 ('object', 0.06282722513089005, 191)
25 ('log4j', 0.5612903225806452, 155)
26 ('properties', 0.36521

In [99]:
df = pd.DataFrame(highest, columns=['Term','Ratio','Size'])

In [100]:
df.sort_values(by='Ratio')[80:]

,Term,Ratio,Size
57,difference,0.439716,282
21,intellij,0.446809,235
81,difference,0.446903,226
61,date,0.459834,361
97,switch,0.461538,130
31,hashcode,0.470149,134
44,icon,0.480769,52
95,array,0.487113,388
87,inputstream,0.506849,146
40,constructor,0.514563,206


In [101]:
df.sort_values(by='Size')[80:]

,Term,Ratio,Size
71,maven,0.211039,616
79,list,0.292528,629
8,int,0.099846,651
6,:,0.067365,668
56,java,0.066568,676
92,xml,0.069016,681
18,file,0.342216,713
36,class,0.272605,741
80,application,0.108322,757
19,in,0.058518,769
